In [4]:
%reload_ext autoreload
%autoreload
import pickle, os, json
import torch
from pathlib import Path, PureWindowsPath, PurePosixPath
from util.tools import load_config
import pandas as pd

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [6]:
config_folder = "config/"
config = load_config(config_folder)

In [15]:
reports_folder_name = config.programsettings.REPORTS_DIR
data_folder = Path(reports_folder_name)
list_results_files = list(data_folder.glob('**/multi_model_*.pkl'))

all_results = []
for results_file in list_results_files:
    print(results_file)
    with open(results_file,"rb") as f:
        lst_results = pickle.load(f)
    for results in lst_results:
        if (len(results[0]) > 50):
            results_json = json.loads(results[0])
            max_seq = results_json["hyperparams"]["MAX_SEQ_LENGTH"]
            lr = results_json["hyperparams"]["LEARNING_RATE"]
            train_batch = results_json["hyperparams"]["TRAIN_BATCH_SIZE"]
            model_name = results_json["programsettings"]["MODEL_NAME"]
        else:
            max_seq = 0
            lr = 0
            train_batch = 0
            max_seq = ""
            model_name = ""
            
        all_results.append([results_file, model_name,lr, train_batch,max_seq,results[1], results[2], results[3], results[4], results[5],results[6] ])

columns = ['file_name','model_name', 'lr', 'train_batch_size','train_max_seq','train_loss', 'dev_loss', 'train_mcc', 'train_f1_score','dev_mcc','dev_f1_score']

df_results = pd.DataFrame(all_results, columns = columns)
pd.set_option('display.max_colwidth', 100)
df_results

reports\re\multi_model_experiment_results_2020-11-15 15_43_01_942702.pkl
reports\re\multi_model_experiment_results_2020-11-15 18_33_46_015615.pkl
reports\re\multi_model_experiment_results_2020-11-16 10_28_19_020854.pkl
reports\re\multi_model_experiment_results_2020-11-20 23_15_57_501227.pkl
reports\re\multi_model_experiment_results_2020-11-21 08_38_53_855089.pkl
reports\re\multi_model_experiment_results_2020-11-21 10_03_19_501249.pkl
reports\re\multi_model_experiment_results_2020-11-22 10_44_11_756583.pkl
reports\re\multi_model_experiment_results_256_5epochs.pkl
reports\re\multi_model_experiment_results_256_5epochs_binary.pkl
reports\re\multi_model_experiment_results_256_5epochs_weighted.pkl
reports\re\multi_model_experiment_results_5epochs.pkl
reports\re\multi_model_experiment_results_CNN.pkl
reports\re\multi_model_experiment_results_CNN_wider.pkl


,file_name,model_name,lr,train_batch_size,train_max_seq,train_loss,dev_loss,train_mcc,train_f1_score,dev_mcc,dev_f1_score
0,reports\re\multi_model_experiment_results_2020-11-15 15_43_01_942702.pkl,,0.000000,0,,2.294667,2.288867,0.004175,0.110000,-0.048113,0.020000
1,reports\re\multi_model_experiment_results_2020-11-15 15_43_01_942702.pkl,,0.000000,0,,2.233493,2.232523,-0.031125,0.050000,-0.058987,0.520000
2,reports\re\multi_model_experiment_results_2020-11-15 18_33_46_015615.pkl,,0.000000,0,,2.041215,2.281210,-0.003173,0.070633,-0.014653,0.040475
3,reports\re\multi_model_experiment_results_2020-11-15 18_33_46_015615.pkl,,0.000000,0,,2.055447,2.839655,0.024098,0.133937,0.011763,0.061364
4,reports\re\multi_model_experiment_results_2020-11-16 10_28_19_020854.pkl,,0.000000,0,,1.491616,2.126475,0.431070,0.405018,0.181686,0.160860
5,reports\re\multi_model_experiment_results_2020-11-16 10_28_19_020854.pkl,,0.000000,0,,1.858239,2.370378,0.256408,0.173122,0.085593,0.056627
6,reports\re\multi_model_experiment_results_2020-11-20 23_15_57_501227.pkl,BioBERT_fc,0.000010,12,128,2.293831,2.218083,-0.074696,0.042834,0.030233,0.088604
7,reports\re\multi_model_experiment_results_2020-11-21 08_38_53_855089.pkl,BioBERT_fc,0.000010,12,128,2.315400,2.255720,0.012113,0.053352,-0.130368,0.053024
8,reports\re\multi_model_experiment_results_2020-11-21 08_38_53_855089.pkl,BioBERT_fc,0.000008,12,128,2.221329,2.343081,-0.057621,0.045866,-0.024152,0.023556
9,reports\re\multi_model_experiment_results_2020-11-21 08_38_53_855089.pkl,BioBERT_fc,0.000013,12,128,2.230306,2.363903,0.035066,0.063515,-0.020312,0.043442


In [ ]:
[str(config), train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               dev_label_ids, dev_preds,train_label_ids,train_preds

10